# Disclaimer:

### The data were collected from the Taiwan Economic Journal for the years 1999to 2009.Company bankruptcy was defined based on the business regulations of the Taiwan Stock Exchange.

# Task:

#### You will use the Taiwanese Bankruptcy Prediction data set. The dataset correspondsto a classification problem on which you need to make predictions on the basis of whetherthe company can have a risk to bankrupt considering features in the dataset. You are asked to use this data to build Bagging Classifier and Boosting Classifiers model.

In [1]:
#ignore warnings
import warnings
warnings.filterwarnings('ignore')

#importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#display all columns
pd.set_option('display.max_columns', None)

from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.metrics import f1_score,precision_score,ConfusionMatrixDisplay,classification_report

# Data Understanting and Processing

In [2]:
#importing dataframe
df = pd.read_csv('data.csv')
df.head()

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,Continuous interest rate (after tax),Operating Expense Rate,Research and development expense rate,Cash flow rate,Interest-bearing debt interest rate,Tax rate (A),Net Value Per Share (B),Net Value Per Share (A),Net Value Per Share (C),Persistent EPS in the Last Four Seasons,Cash Flow Per Share,Revenue Per Share (Yuan ¥),Operating Profit Per Share (Yuan ¥),Per Share Net profit before tax (Yuan ¥),Realized Sales Gross Profit Growth Rate,Operating Profit Growth Rate,After-tax Net Profit Growth Rate,Regular Net Profit Growth Rate,Continuous Net Profit Growth Rate,Total Asset Growth Rate,Net Value Growth Rate,Total Asset Return Growth Rate Ratio,Cash Reinvestment %,Current Ratio,Quick Ratio,Interest Expense Ratio,Total debt/Total net worth,Debt ratio %,Net worth/Assets,Long-term fund suitability ratio (A),Borrowing dependency,Contingent liabilities/Net worth,Operating profit/Paid-in capital,Net profit before tax/Paid-in capital,Inventory and accounts receivable/Net value,Total Asset Turnover,Accounts Receivable Turnover,Average Collection Days,Inventory Turnover Rate (times),Fixed Assets Turnover Frequency,Net Worth Turnover Rate (times),Revenue per person,Operating profit per person,Allocation rate per person,Working Capital to Total Assets,Quick Assets/Total Assets,Current Assets/Total Assets,Cash/Total Assets,Quick Assets/Current Liability,Cash/Current Liability,Current Liability to Assets,Operating Funds to Liability,Inventory/Working Capital,Inventory/Current Liability,Current Liabilities/Liability,Working Capital/Equity,Current Liabilities/Equity,Long-term Liability to Current Assets,Retained Earnings to Total Assets,Total income/Total expense,Total expense/Assets,Current Asset Turnover Rate,Quick Asset Turnover Rate,Working capitcal Turnover Rate,Cash Turnover Rate,Cash Flow to Sales,Fixed Assets to Assets,Current Liability to Liability,Current Liability to Equity,Equity to Long-term Liability,Cash Flow to Total Assets,Cash Flow to Liability,CFO to Assets,Cash Flow to Equity,Current Liability to Current Assets,Liability-Assets Flag,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,0.780985,1.256969e-04,0.0,0.458143,0.000725,0.0,0.147950,0.147950,0.147950,0.169141,0.311664,0.017560,0.095921,0.138736,0.022102,0.848195,0.688979,0.688979,0.217535,4.980000e+09,0.000327,0.263100,0.363725,0.002259,0.001208,0.629951,0.021266,0.207576,0.792424,0.005024,0.390284,0.006479,0.095885,0.137757,0.398036,0.086957,0.001814,0.003487,1.820926e-04,1.165007e-04,0.032903,0.034164,0.392913,0.037135,0.672775,0.166673,0.190643,0.004094,0.001997,1.473360e-04,0.147308,0.334015,0.276920,0.001036,0.676269,0.721275,0.339077,0.025592,0.903225,0.002022,0.064856,7.010000e+08,6.550000e+09,0.593831,4.580000e+08,0.671568,0.424206,0.676269,0.339077,0.126549,0.637555,0.458609,0.520382,0.312905,0.118250,0,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,0.781506,2.897851e-04,0.0,0.461867,0.000647,0.0,0.182251,0.182251,0.182251,0.208944,0.318137,0.021144,0.093722,0.169918,0.022080,0.848088,0.689693,0.689702,0.217620,6.110000e+09,0.000443,0.264516,0.376709,0.006016,0.004039,0.635172,0.012502,0.171176,0.828824,0.005059,0.376760,0.005835,0.093743,0.168962,0.397725,0.064468,0.001286,0.004917,9.360000e+09,7.190000e+08,0.025484,0.0068

In [3]:
#shape of dataframe
df.shape

(6819, 96)

In [4]:
#checking for objects
for col in df.columns:
    if df[col].dtype == 'object':
        print(col)

In [5]:
#checking for. null values
df.isnull().any().sum()

0

In [6]:
#checking for duplicates values
df.duplicated().sum()

0

In [7]:
#describe all data
df.describe().T
#data seem like normal

,count,mean,std,min,25%,50%,75%,max
Bankrupt?,6819.0,0.032263,0.176710,0.0,0.000000,0.000000,0.000000,1.0
ROA(C) before interest and depreciation before interest,6819.0,0.505180,0.060686,0.0,0.476527,0.502706,0.535563,1.0
ROA(A) before interest and % after tax,6819.0,0.558625,0.065620,0.0,0.535543,0.559802,0.589157,1.0
ROA(B) before interest and depreciation after tax,6819.0,0.553589,0.061595,0.0,0.527277,0.552278,0.584105,1.0
Operating Gross Margin,6819.0,0.607948,0.016934,0.0,0.600445,0.605997,0.613914,1.0
...,...,...,...,...,...,...,...,...
Liability to Equity,6819.0,0.280365,0.014463,0.0,0.276944,0.278778,0.281449,1.0
Degree of Financial Leverage (DFL),6819.0,0.027541,0.015668,0.0,0.026791,0.026808,0.026913,1.0
Interest Coverage Ratio (Interest expense to EBIT),6819.0,0.565358,0.013214,0.0,0.565158,0.565252,0.565725,1.0
Net Income Flag,6819.0,1.000000,0.000000,1.0,1.000000,1.000000,1.000000,1.0


In [8]:
#replacing feature names 
df.columns = df.columns.str.replace(r"[?%¥()]","")
df.columns = df.columns.str.replace(r"[/ ]","_")

In [9]:
#checking for value counts that is 1
for col in df.columns:
    if len(df[col].value_counts()) == 1:
        print(col)
#drop the column that has 1 unique value
df.drop('_Net_Income_Flag',inplace=True,axis=1)

_Net_Income_Flag


In [10]:
df.corr()
#some columns has low correlation(<0.05) and some columns is already same,it can be happened to the Multicollinearity
#let's get delete them

,Bankrupt,_ROAC_before_interest_and_depreciation_before_interest,_ROAA_before_interest_and__after_tax,_ROAB_before_interest_and_depreciation_after_tax,_Operating_Gross_Margin,_Realized_Sales_Gross_Margin,_Operating_Profit_Rate,_Pre-tax_net_Interest_Rate,_After-tax_net_Interest_Rate,_Non-industry_income_and_expenditure_revenue,_Continuous_interest_rate_after_tax,_Operating_Expense_Rate,_Research_and_development_expense_rate,_Cash_flow_rate,_Interest-bearing_debt_interest_rate,_Tax_rate_A,_Net_Value_Per_Share_B,_Net_Value_Per_Share_A,_Net_Value_Per_Share_C,_Persistent_EPS_in_the_Last_Four_Seasons,_Cash_Flow_Per_Share,_Revenue_Per_Share_Yuan_,_Operating_Profit_Per_Share_Yuan_,_Per_Share_Net_profit_before_tax_Yuan_,_Realized_Sales_Gross_Profit_Growth_Rate,_Operating_Profit_Growth_Rate,_After-tax_Net_Profit_Growth_Rate,_Regular_Net_Profit_Growth_Rate,_Continuous_Net_Profit_Growth_Rate,_Total_Asset_Growth_Rate,_Net_Value_Growth_Rate,_Total_Asset_Return_Growth_Rate_Ratio,_Cash_Reinvestment_,_Current_Ratio,_Quick_Ratio,_Interest_Expense_Ratio,_Total_debt_Total_net_worth,_Debt_ratio_,_Net_worth_Assets,_Long-term_fund_suitability_ratio_A,_Borrowing_dependency,_Contingent_liabilities_Net_worth,_Operating_profit_Paid-in_capital,_Net_profit_before_tax_Paid-in_capital,_Inventory_and_accounts_receivable_Net_value,_Total_Asset_Turnover,_Accounts_Receivable_Turnover,_Average_Collection_Days,_Inventory_Turnover_Rate_times,_Fixed_Assets_Turnover_Frequency,_Net_Worth_Turnover_Rate_times,_Revenue_per_person,_Operating_profit_per_person,_Allocation_rate_per_person,_Working_Capital_to_Total_Assets,_Quick_Assets_Total_Assets,_Current_Assets_Total_Assets,_Cash_Total_Assets,_Quick_Assets_Current_Liability,_Cash_Current_Liability,_Current_Liability_to_Assets,_Operating_Funds_to_Liability,_Inventory_Working_Capital,_Inventory_Current_Liability,_Current_Liabilities_Liability,_Working_Capital_Equity,_Current_Liabilities_Equity,_Long-term_Liability_to_Current_Assets,_Retained_Earnings_to_Total_Assets,_Total_income_Total_expense,_Total_expense_Assets,_Current_Asset_Turnover_Rate,_Quick_Asset_Turnover_Rate,_Working_capitcal_Turnover_Rate,_Cash_Turnover_Rate,_Cash_Flow_to_Sales,_Fixed_Assets_to_Assets,_Current_Liability_to_Liability,_Current_Liability_to_Equity,_Equity_to_Long-term_Liability,_Cash_Flow_to_Total_Assets,_Cash_Flow_to_Liability,_CFO_to_Assets,_Cash_Flow_to_Equity,_Current_Liability_to_Current_Assets,_Liability-Assets_Flag,_Net_Income_to_Total_Assets,_Total_assets_to_GNP_price,_No-credit_Interval,_Gross_Profit_to_Sales,_Net_Income_to_Stockholder's_Equity,_Liability_to_Equity,_Degree_of_Financial_Leverage_DFL,_Interest_Coverage_Ratio_Interest_expense_to_EBIT,_Equity_to_Liability
Bankrupt,1.000000,-0.260807,-0.282941,-0.273051,-0.100043,-0.099445,-0.000230,-0.008517,-0.008857,-0.016593,-0.008395,-0.006083,-0.024232,-0.072356,-0.023063,-0.109706,-0.165399,-0.165465,-0.164784,-0.219560,-0.077516,-0.004692,-0.142051,-0.201395,-0.000458,-0.015168,-0.037783,-0.036820,-0.009401,-0.044431,0.065329,-0.016858,-0.051345,-0.002211,0.025058,-0.002681,0.012314,0.250161,-0.250161,0.016920,0.176543,0.070455,-0.141111,-0.207857,0.075278,-0.067915,-0.004754,-0.006556,-0.001376,0.072818,0.021089,0.039718,-0.092842,0.002829,-0.193083,-0.086382,-0.044823,-0.100130,-0.003823,0.077921,0.194494,-0.077082,-0.001906,0.000822,-0.020809,-0.147221,0.153828,0.000778,-0.217779,-0.007137,0.139049,0.011929,0.025814,-0.002894,-0.018035,0.000479,0.066328,-0.020809,0.153828,0.139014,-0.070456,-0.043125,-0.115383,-0.058563,0.171306,0.139212,-0.315457,0.035104,-0.005547,-0.100044,-0.180987,0.166812,0.010508,-0.005509,-0.083048
_ROAC_before_interest_and_depreciation_before_interest,-0.260807,1.000000,0.940124,0.986849,0.334719,0.332755,0.035725,0.053419,0.049222,0.020501,0.051328,0.066869,0.106461,0.323482,0.048882,0.250761,0.505580,0.505407,0.505281,0.775006,0.379839,-0.015932,0.687201,0.750564,0.000591,0.036511,0.115083,0.115040,0.025234,0.019635,-0.021930,0.079906,0.296158,0.013196,-

In [11]:
#drop some unsignificant columns
df.drop(columns=['_ROAC_before_interest_and_depreciation_before_interest','_Realized_Sales_Gross_Margin',
                 '_ROAB_before_interest_and_depreciation_after_tax','_Operating_Profit_Rate','_After-tax_net_Interest_Rate',
                '_Non-industry_income_and_expenditure_revenue','_Continuous_interest_rate_after_tax','_Current_Ratio',
                 '_Operating_Expense_Rate','_Interest-bearing_debt_interest_rate','_Net_Value_Per_Share_B',
                '_Net_Value_Per_Share_A','_Net_Value_Per_Share_B','_Revenue_Per_Share_Yuan_',
                 '_Realized_Sales_Gross_Profit_Growth_Rate','_After-tax_Net_Profit_Growth_Rate',
                '_Continuous_Net_Profit_Growth_Rate','_Total_Asset_Return_Growth_Rate_Ratio','_Quick_Ratio',
                '_Interest_Expense_Ratio','_Total_debt_Total_net_worth','_Accounts_Receivable_Turnover',
                '_Average_Collection_Days','_Inventory_Turnover_Rate_times','_Net_Worth_Turnover_Rate_times',
                '_Revenue_per_person','_Allocation_rate_per_person','_Quick_Assets_Current_Liability',
                '_Inventory_Working_Capital','_Inventory_Current_Liability','_Long-term_Liability_to_Current_Assets',
                '_Total_income_Total_expense','_Quick_Asset_Turnover_Rate','_Current_Asset_Turnover_Rate',
                '_Working_capitcal_Turnover_Rate','_Quick_Asset_Turnover_Rate','_Cash_Turnover_Rate',
                '_Fixed_Assets_to_Assets','_Current_Liability_to_Liability','_Cash_Flow_to_Liability',
                 '_Total_assets_to_GNP_price','_Degree_of_Financial_Leverage_DFL',
                 '_Interest_Coverage_Ratio_Interest_expense_to_EBIT'],axis=1,inplace=True)
print(f"{95-len(df.columns)} columnS have been deleted")

41 columnS have been deleted


In [12]:
df

,Bankrupt,_ROAA_before_interest_and__after_tax,_Operating_Gross_Margin,_Pre-tax_net_Interest_Rate,_Research_and_development_expense_rate,_Cash_flow_rate,_Tax_rate_A,_Net_Value_Per_Share_C,_Persistent_EPS_in_the_Last_Four_Seasons,_Cash_Flow_Per_Share,_Operating_Profit_Per_Share_Yuan_,_Per_Share_Net_profit_before_tax_Yuan_,_Operating_Profit_Growth_Rate,_Regular_Net_Profit_Growth_Rate,_Total_Asset_Growth_Rate,_Net_Value_Growth_Rate,_Cash_Reinvestment_,_Debt_ratio_,_Net_worth_Assets,_Long-term_fund_suitability_ratio_A,_Borrowing_dependency,_Contingent_liabilities_Net_worth,_Operating_profit_Paid-in_capital,_Net_profit_before_tax_Paid-in_capital,_Inventory_and_accounts_receivable_Net_value,_Total_Asset_Turnover,_Fixed_Assets_Turnover_Frequency,_Operating_profit_per_person,_Working_Capital_to_Total_Assets,_Quick_Assets_Total_Assets,_Current_Assets_Total_Assets,_Cash_Total_Assets,_Cash_Current_Liability,_Current_Liability_to_Assets,_Operating_Funds_to_Liability,_Current_Liabilities_Liability,_Working_Capital_Equity,_Current_Liabilities_Equity,_Retained_Earnings_to_Total_Assets,_Total_expense_Assets,_Cash_Flow_to_Sales,_Current_Liability_to_Equity,_Equity_to_Long-term_Liability,_Cash_Flow_to_Total_Assets,_CFO_to_Assets,_Cash_Flow_to_Equity,_Current_Liability_to_Current_Assets,_Liability-Assets_Flag,_Net_Income_to_Total_Assets,_No-credit_Interval,_Gross_Profit_to_Sales,_Net_Income_to_Stockholder's_Equity,_Liability_to_Equity,_Equity_to_Liability
0,1,0.424389,0.601457,0.796887,0.000000e+00,0.458143,0.000000,0.147950,0.169141,0.311664,0.095921,0.138736,0.848195,0.688979,4.980000e+09,0.000327,0.363725,0.207576,0.792424,0.005024,0.390284,0.006479,0.095885,0.137757,0.398036,0.086957,1.165007e-04,0.392913,0.672775,0.166673,0.190643,0.004094,1.473360e-04,0.147308,0.334015,0.676269,0.721275,0.339077,0.903225,0.064856,0.671568,0.339077,0.126549,0.637555,0.520382,0.312905,0.118250,0,0.716845,0.622879,0.601453,0.827890,0.290202,0.016469
1,1,0.538214,0.610235,0.797380,0.000000e+00,0.461867,0.000000,0.182251,0.208944,0.318137,0.093722,0.169918,0.848088,0.689702,6.110000e+09,0.000443,0.376709,0.171176,0.828824,0.005059,0.376760,0.005835,0.093743,0.168962,0.397725,0.064468,7.190000e+08,0.391590,0.751111,0.127236,0.182419,0.014948,1.383910e-03,0.056963,0.341106,0.308589,0.731975,0.329740,0.931065,0.025516,0.671570,0.329740,0.120916,0.641100,0.567101,0.314163,0.047775,0,0.795297,0.623652,0.610237,0.839969,0.283846,0.020794
2,1,0.499019,0.601450,0.796403,2.550000e+07,0.458521,0.000000,0.193713,0.180581,0.307102,0.092338,0.142803,0.848094,0.689470,7.280000e+09,0.000396,0.368913,0.207516,0.792484,0.005100,0.379093,0.006562,0.092318,0.148036,0.406580,0.014993,2.650000e+09,0.381968,0.829502,0.340201,0.602806,0.000991,5.340000e+09,0.098162,0.336731,0.446027,0.742729,0.334777,0.909903,0.021387,0.671571,0.334777,0.117922,0.642765,0.538491,0.314515,0.025346,0,0.774670,0.623841,0.601449,0.836774,0.290189,0.016474
3,1,0.451265,0.583541,0.796967,0.000000e+00,0.465705,0.000000,0.154187,0.193722,0.321674,0.077762,0.148603,0.848005,0.689110,4.880000e+09,0.000382,0.384077,0.151465,0.848535,0.005047,0.379743,0.005366,0.077727,0.147561,0.397925,0.089955,9.150000e+09,0.378497,0.725754,0.161575,0.225815,0.018851,1.010646e-03,0.098715,0.348716,0.615848,0.729825,0.331509,0.906902,0.024161,0.671519,0.331509,0.120760,0.579039,0.604105,0.302382,0.067250,0,0.739555,0.622929,0.583538,0.834697,0.281721,0.023982
4,1,0.538432,0.598783,0.797366,0.000000e+00,0.462746,0.000000,0.167502,0.212537,0.319162,0.096898,0.168412,0.848258,0.689697,5.510000e+09,0.000439,0.379690,0.106509,0.893491,0.005303,0.375025,0.006624,0.096927,0.167461,0.400079,0.175412,2.935211e-04,0.394371,0.751822,0.260330,0.358380,0.014161,6.804636e-04,0.110195,0.344639,0.975007,0.732000,0.330726,0.913850,0.026385,0.671563,0.330726,0.110933,0.622374,0.578469,0.311567,0.047725,0,0.795016,0.623521,0.598782,0.839973,0.278514,0.035490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

# Visualizations to get inside from the data